<a href="https://colab.research.google.com/github/41140059/task/blob/main/Padma_sri_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data processing


In [ ]:
df = pd.read_csv('/kaggle/input/weather-prediction/seattle-weather.csv')
df.head()

In [ ]:
df.drop(columns=['date'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df.head(10)

In [ ]:
# Define a mapping dictionary
mapping = {'drizzle': 0, 'fog': 1, 'rain': 2, 'snow': 3, 'sun': 4}

# Apply the mapping to the 'Category' column
df['weather'] = df['weather'].map(mapping)
print(df)

cleaning outliers

In [ ]:
column = ['precipitation','temp_max','temp_min', 'wind']
df[column].describe()

In [ ]:
for cols in column:
  sns.boxplot(x=df[cols])
  plt.show()

k means clustering(elbow method)

In [ ]:
inertias = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42,  n_init=10)
    kmeans.fit(df)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), inertias, marker='o')
plt.xlabel('Cluster')
plt.ylabel('Inersia')
plt.title('Elbow Method')
plt.show()

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df)

n_clusters = 5

kmeans = KMeans(n_clusters=n_clusters, random_state=42,  n_init=10)
df['cluster'] = kmeans.fit_predict(scaled_features)

df.to_csv('data_baru.csv', index=False)

In [ ]:
df_clean = pd.read_csv("data_baru.csv")
df_clean.head(100)

split database

In [ ]:
X = df.drop(['cluster'], axis=1)
y = df['cluster']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

model development


In [ ]:
start_time = time.time()

In [ ]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
grid_search = GridSearchCV(knn, param_grid, cv=4)
grid_search.fit(X_train, y_train)

print("Best Parameter", grid_search.best_params_)

best_knn = grid_search.best_estimator_

best_knn.fit(X_train, y_train)
y_pred = best_knn.predict(X_test)

result

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

precision = precision_score(y_test, y_pred, average='weighted')
print("Precicion:", precision)

recall = recall_score(y_test, y_pred, average='weighted')
print("Recall:", recall)

f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

In [ ]:
end_time = time.time()

process_time = end_time - start_time

print(f"Time run: {process_time} second")

Conclusion: The KNN-based weather prediction model achieved high accuracy, demonstrating the algorithm's effectiveness for this task. Future work includes exploring more advanced models like Random Forest and Gradient Boosting to further improve prediction accuracy.